# Reshaping and Pivoting

## 1. Reshaping with Hierarchical Indexing

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))

In [3]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [4]:
#stack(): DF => Series with hierarchical index
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [5]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [6]:
#by default the inner level, passing level number unstacks the desired levels
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [7]:
#alternatively
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


### 1.1 MIssing Data

In [8]:
s1 = pd.Series([0, 1, 2, 3], index=list('abcd'))
s2 = pd.Series([4, 5, 6], index=list('cde'))

In [9]:
data_2 = pd.concat([s1, s2], keys=['one', 'two'])
data_2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [10]:
data_2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [11]:
data_2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [12]:
data_2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

## 1.2 DataFrame

In [13]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [14]:
frame = pd.DataFrame({'left': result, 'right': result + 5},
                     columns=pd.Index(['left', 'right'], name='side'))

In [15]:
frame

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [16]:
frame.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [17]:
frame.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## 2. Pivoting
* pivot() = another type of unstack, commonly used to pivot MySQL database data
* melt() = inverse of pivot()

### 2.1 `Wide` to `Long` format

In [18]:
frame = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                     'A': [1, 2, 3],
                     'B': [4, 5, 6],
                     'C': [7, 8, 9]})

In [19]:
frame

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [20]:
melted = pd.melt(frame, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [21]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [22]:
#Back to numeric indexing the rows
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [23]:
#a subset of columns
pd.melt(frame, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [24]:
#without identifiers
pd.melt(frame, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9
